In [1]:
import os
os.chdir('..')
# os.chdir('eval')
# os.chdir('../eval')
%load_ext autoreload

import hashlib
import time
import random
import json
import openai
import transformers
import fnmatch
import datasets
import pathlib
import torch

from functools import cache
from collections import Counter
from datasets import load_dataset
from warnings import warn
from abc import abstractmethod, ABC
from eval import tasks
from eval.tasks import utils
from eval.utils import TokenizedDataset, complete_code
from eval.tasks.utils import evaluate, convert_to_nltk_rep
# from eval.generation import parallel_generations
from diskcache import Cache
from concurrent.futures import ThreadPoolExecutor


from torch.utils.data.dataloader import DataLoader
from transformers import StoppingCriteria, StoppingCriteriaList
from accelerate.utils import set_seed

from accelerate import Accelerator, DeepSpeedPlugin
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser

from eval import tasks
from eval.generation import parallel_generations
from eval.args import RunnerArguments, HFArguments, OAIArguments, GenerationArguments
# from eval.evaluator import HFEvaluator, OAIEvaluator
from eval.tasks import ALL_TASKS, TASK_REGISTRY


transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()

/home/gatum/.conda/envs/linc_modify/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-09 11:15:04,789] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


FOL (Usage Example)

In [2]:
os.getcwd()

'/home/gatum/Projects/Neurosymbolic-AI/linc2/linc_modify'

In [3]:
premises = [ "∀x (RomanceLanguage(x) → IndoEuropeanLanguage(x))", "∀x (RomanceLanguage(x) → MemberOf(x, romancefamily))", "∀x ∀y ∀z ((MemberOf(x, z) ∧ MemberOf(y, z)) → (Related(x, y) ∧ Related(y, x)))", "RomanceLanguage(french) ∧ RomanceLanguage(spanish)", "Related(german, spanish)", "∀x (¬Related(basque, x))" ]
conclusion = 'IndoEuropeanLanguage(french)'

premises_prep = [utils.convert_to_nltk_rep(p) for p in premises]
conclusion_prep = utils.convert_to_nltk_rep(conclusion)

utils.evaluate(premises_prep, conclusion_prep)

'True'

FOL FROM LLAMA + ex_ctxs ONLY

In [42]:
premises = '''TEXT: All dispensable things are environment-friendly.
FOL: all x. (Dispensable(x) -> EnvironmentFriendly(x))
TEXT: All woodware is dispensable.
FOL: all x. (Woodware(x) -> Dispensable(x))
TEXT: All paper is woodware.
FOL: all x. (Paper(x) -> Woodware(x))
TEXT: No good things are bad.
FOL: all x. (Good(x) -> -Bad(x))
TEXT: All environment-friendly things are good.
FOL: all x. (EnvironmentFriendly(x) -> Good(x))
TEXT: A worksheet is either paper or is environment-friendly.
FOL: ((Paper(Worksheet) & -EnvironmentFriendly(Worksheet)) | (-Paper(Worksheet) & EnvironmentFriendly(Worksheet)))
TEXT: A worksheet is not dispensable.
FOL: -Dispensable(Worksheet)

TEXT: All athletes are good at sports.
FOL: all x. (Athlete(x) -> GoodAtSports(x))
TEXT: All Olympic gold medal winners are good athletes.
FOL: all x. (OlympicGoldMedalWinner(x) -> Athlete(x))
TEXT: No scientists are good at sports.
FOL: all x. (Scientist(x) -> -GoodAtSports(x))
TEXT: All Nobel laureates are scientists.
FOL: all x. (NobelLaureate(x) -> Scientist(x))
TEXT: Amy is good at sports or Amy is an Olympic gold medal winner.
FOL: GoodAtSports(Amy) | OlympicGoldMedalWinner(Amy)
TEXT: If Amy is not a Nobel laureate, then Amy is not an Olympic gold medal winner.
FOL: -NobelLaureate(Amy) -> -OlympicGoldMedalWinner(Amy)
TEXT: If Amy is not an Olympic gold medal winner, then Amy is a Nobel laureate.
FOL: -OlympicGoldMedalWinner(Amy) -> NobelLaureate(Amy)

TEXT: A La Liga soccer team ranks higher than another if it receives more points.
FOL: all x. all y. (LaLiga(x) & LaLiga(y) & MorePoints(x, y) -> HigherRank(x, y))
TEXT: If two La Liga soccer teams recieve the same points, the team which recieves more points from the games between the two teams ranks higher.
FOL: all x. all y. (LaLiga(x) & LaLiga(y) & -MorePoints(x, y) & -MorePoints(y, x) & MorePointsInGameBetween(x, y) -> HigherRank(x, y))
TEXT: Real Madrid and Barcelona are both La Liga soccer teams.
FOL: LaLiga(RealMadrid) & LaLiga(Barcelona)
TEXT: In La Liga 2021-2022, Real Madrid recieves 86 points and Barcelon recieves 73 points.
FOL: MorePoints(RealMadrid, Barcelona)
TEXT: In La Liga 2021-2022, Real Madrid and Barcelona both recieve 3 points from the games between them.
FOL: -MorePointsInGameBetween(RealMadrid, Barcelona) & -MorePointsInGameBetween(Barcelona, RealMadrid)
TEXT: In La Liga 2021-2022, Real Madrid ranks higher than Barcelona.
FOL: HigherRank(RealMadrid, Barcelona)

TEXT: Skateboarding is an action sport that involves riding and performing tricks using a skateboard, as well as a recreational activity, an art form, an entertainment industry job, and a method of transportation.
FOL: Skateboarding(Skateboarding)
TEXT: Originating in the United States, skateboarding has been shaped and influenced by many skateboarders throughout the years.
FOL: SkateboardingOriginatedInTheUnitedStates(Skateboarding)
TEXT: A 2009 report found that the skateboarding market is worth an estimated $4.8 billion in annual revenue, with 11.08 million active skateboarders in the world.
FOL: SkateboardingMarketWorth4Point8Billion(Skateboarding)
TEXT: In the third quarter of 2021, Delivery Hero processed 791 million orders—that equals a year-on-year growth of 52%.
FOL: DeliveryHeroProcessed791MillionOrders(In2021)

TEXT: Skateboarder was primarily a digital skateboarding publication that produces a limited run of hard copy versions that are sold in skateboard shops.
FOL: SkateboarderWasAPrimaryDigitalPublication(Skateboarder)
TEXT: The publication was the United States' first skateboarding magazine.
FOL: SkateboarderWasTheFirstSkateboardingMagazine(Skateboarder)
TEXT: In general, and in contrast to recent legal precedents in Canada and Australia, Delivery Hero does not classify these couriers as employees.
FOL: DeliveryHeroDoesNotClassifyCouriersAsEmployees(DeliveryHero)
TEXT: A delivery or ball in cricket is a single action of bowling a cricket ball toward the batter.
FOL: DeliveryOrBallInCricket(SingleActionOfBowlingACricketBall)

TEXT: Cabell Calloway III (December 25, 1907 – November 18, 1994) was an American jazz singer and bandleader.
FOL: CabellCallowayIIIWasAnAmericanJazzSingerAndBandleader(CabellCallowayIII)
TEXT: Death Cab for Cutie (commonly abbreviated to DCFC or Death Cab) is an American rock band formed in Bellingham, Washington, in 1997.
FOL: DeathCabForCutieIsAnAmericanRockBand(FormedIn1997)

TEXT: Skateboarder is the man by the bags awaiting.
FOL: SkateboarderIsTheManByTheBagsAwaiting(Skateboarder)'''

In [45]:
premises = """

<EVALUATE>
TEXT:	Skateboarding is an action sport that involves riding and performing tricks using a skateboard, as well as a recreational activity, an art form, an entertainment industry job, and a method of transportation.
FOL:	ActionSport(Skateboarding) & RecreationalActivity(Skateboarding) & ArtForm(Skateboarding) & EntertainmentIndustryJob(Skateboarding) & TransportationMethod(Skateboarding)
TEXT: Originating in the United States, skateboarding has been shaped and influenced by many skateboarders throughout the years.
FOL: OriginatedIn(Skateboarding, UnitedStates) & InfluencedByMany(Skateboarding, Skateboarders)

TEXT: A 2009 report found that the skateboarding market is worth an estimated $4.8 billion in annual revenue, with 11.08 million active skateboarders in the world.
FOL: MarketWorth(Skateboarding, 4.8Billion) & ActiveSkateboarders(World, 11.08Million)

TEXT: In 2016, it was announced that skateboarding would be represented at the 2020 Summer Olympics in Tokyo, for both male and female teams.
FOL: RepresentedAt(Skateboarding, SummerOlympics2020)

TEXT: Since the 1970s, skateparks have been constructed specifically for use by skateboarders, freestyle BMXers, aggressive skaters, and more recently, scooters.
FOL: ConstructedFor(Skateparks, [Skateboarders, FreestyleBMXers, AggressiveSkaters, Scooters])

TEXT: Skateboarding has become controversial in areas in which the activity, although legal, has damaged curbs, stoneworks, steps, benches, plazas, and parks.
FOL: ControversialInAreas(Skateboarding, [Curbs, Stoneworks, Steps, Benches, Plazas, Parks])

TEXT: Skateboarder was primarily a digital skateboarding publication that produces a limited run of hard copy versions that are sold in skateboard shops.
FOL: PublicationType(Skateboarder, Digital) & SoldIn(Skateboarder, SkateboardShops)

TEXT: The publication was the United States' first skateboarding magazine.
FOL: FirstSkateboardingMagazine(Skateboarder, UnitedStates)

TEXT: In August 2013, its editor was Jaime Owens and its publisher was Jamey Stone.
FOL: Editor(Skateboarder, JaimeOwens, August2013) & Publisher(Skateboarder, JameyStone, August2013)

TEXT: On August 19, 2013, the magazine's owner GrindMedia announced that the publication would cease production on October 15, 2013.
FOL: CeaseProduction(Skateboarder, October15_2013)

TEXT: A skateboard is a type of sports equipment used for skateboarding.
FOL: SportsEquipment(Skateboard) & UsedFor(Skateboard, Skateboarding)

TEXT: The skateboard moves by pushing with one foot while the other foot remains balanced on the board, or by pumping one's legs in structures such as a bowl or half pipe.
FOL: MovesBy(Skateboard, [PushingOneFoot, PumpingLegs])

TEXT: A skateboard can also be used by standing on the deck while on a downward slope and allowing gravity to propel the board and the rider.
FOL: UsedBy(Skateboard, [StandingOnDeck, DownwardSlope])

TEXT: If the rider's leading foot is their left foot, they are said to ride "regular". Conversely, they are said to ride "goofy" if their leading foot is their right foot.
FOL: RideStyle(LeftFoot, Regular) & RideStyle(RightFoot, Goofy)

TEXT: The two main types of skateboards are the longboard and shortboard.
FOL: MainTypesOfSkateboards([Longboard, Shortboard])

TEXT: The shape of the board is also important: the skateboard must be concaved to perform tricks.
FOL: ImportantShape(Skateboard, Concaved)

TEXT: Skateboarder is the man by the bags awaiting.
FOL: ManByBagsAwaiting(Skateboarder)
<EVALUATE/>
"""

In [49]:
premises = '''
TEXT: All skateboarding is an action sport that involves riding and performing tricks using a skateboard, as well as a recreational activity, an art form, an entertainment industry job, and a method of transportation.
FOL: all x. (Skateboarding(x) -> (ActionSport(x) & (Riding(x) & PerformingTricks(x)) & (RecreationalActivity(x) & ArtForm(x) & EntertainmentIndustryJob(x) & MethodOfTransportation(x))))

TEXT: Skateboarding has been shaped and influenced by many skateboarders throughout the years.
FOL: all x. (Skateboarding(x) -> (Shaped(x) & Influenced(x)))

TEXT: Skateboarding has become controversial in areas in which the activity, although legal, has damaged curbs, stoneworks, steps, benches, plazas, and parks.
FOL: all x. (Skateboarding(x) -> (Controversial(x) & Damaged(x)))

TEXT: A skateboard is a type of sports equipment used for skateboarding.
FOL: all x. (Skateboard(x) -> (SportsEquipment(x) & UsedForSkateboarding(x)))

TEXT: The skateboard moves by pushing with one foot while the other foot remains balanced on the board, or by pumping one's legs in structures such as a bowl or half pipe.
FOL: all x. (Skateboard(x) -> (Moves(x) & (Pushing(x) & Balanced(x)) | Pumping(x)))

TEXT: A train (from Old French trahiner, from Latin trahere, "to pull, to draw") is a series of connected vehicles that run along a railway track and transport people or freight.
FOL: all x. (Train(x) -> (SeriesOfConnectedVehicles(x) & RunAlongRailwayTrack(x) & Transport(x)))

TEXT: Trains have their roots in wagonways, which used railway tracks and were powered by horses or pulled by cables.
FOL: all x. (Train(x) -> (RootsInWagonways(x) & UsedRailwayTracks(x) & PoweredByHorsesOrCables(x)))

TEXT: Delivery Hero SE is a German multinational online food ordering and food delivery company based in Berlin, Germany.
FOL: all x. (DeliveryHero(x) -> (GermanMultinational(x) & OnlineFoodOrdering(x) & FoodDelivery(x) & BasedInBerlin(x)))

TEXT: A delivery or ball in cricket is a single action of bowling a cricket ball toward the batter.
FOL: all x. (Delivery(x) -> (SingleAction(x) & Bowling(x) & TowardBatter(x)))

TEXT: Cabell Calloway III (December 25, 1907 – November 18, 1994) was an American jazz singer and bandleader.
FOL: all x. (CabellCalloway(x) -> (American(x) & JazzSinger(x) & Bandleader(x)))

TEXT: Death Cab for Cutie (commonly abbreviated to DCFC or Death Cab) is an American rock band formed in Bellingham, Washington, in 1997.
FOL: all x. (DeathCabForCutie(x) -> (American(x) & RockBand(x) & FormedInBellingham(x) & FormedIn1997(x)))

TEXT: Skateboarder is the man by the bags awaiting.
FOL: Skateboarder(x) -> ManByBagsAwaiting(x)

TEXT: Train is the man by the bags awaiting.
FOL: Train(x) -> ManByBagsAwaiting(x)

TEXT: Delivery is the man by the bags awaiting.
FOL: Delivery(x) -> ManByBagsAwaiting(x)

TEXT: Cab is the man by the bags awaiting.
FOL: Cab(x) -> ManByBagsAwaiting(x)
'''

In [50]:
premises_only = []
for line in premises.splitlines():
    splitted = (line.split('FOL: '))
    if len(splitted) >= 2:
        print(splitted[1])
        premises_only.append(splitted[1])
# print
print(len(premises_only))
conclusion_only = premises_only.pop()

all x. (Skateboarding(x) -> (ActionSport(x) & (Riding(x) & PerformingTricks(x)) & (RecreationalActivity(x) & ArtForm(x) & EntertainmentIndustryJob(x) & MethodOfTransportation(x))))
all x. (Skateboarding(x) -> (Shaped(x) & Influenced(x)))
all x. (Skateboarding(x) -> (Controversial(x) & Damaged(x)))
all x. (Skateboard(x) -> (SportsEquipment(x) & UsedForSkateboarding(x)))
all x. (Skateboard(x) -> (Moves(x) & (Pushing(x) & Balanced(x)) | Pumping(x)))
all x. (Train(x) -> (SeriesOfConnectedVehicles(x) & RunAlongRailwayTrack(x) & Transport(x)))
all x. (Train(x) -> (RootsInWagonways(x) & UsedRailwayTracks(x) & PoweredByHorsesOrCables(x)))
all x. (DeliveryHero(x) -> (GermanMultinational(x) & OnlineFoodOrdering(x) & FoodDelivery(x) & BasedInBerlin(x)))
all x. (Delivery(x) -> (SingleAction(x) & Bowling(x) & TowardBatter(x)))
all x. (CabellCalloway(x) -> (American(x) & JazzSinger(x) & Bandleader(x)))
all x. (DeathCabForCutie(x) -> (American(x) & RockBand(x) & FormedInBellingham(x) & FormedIn1997(x

In [51]:
premises_only, conclusion_only

(['all x. (Skateboarding(x) -> (ActionSport(x) & (Riding(x) & PerformingTricks(x)) & (RecreationalActivity(x) & ArtForm(x) & EntertainmentIndustryJob(x) & MethodOfTransportation(x))))',
  'all x. (Skateboarding(x) -> (Shaped(x) & Influenced(x)))',
  'all x. (Skateboarding(x) -> (Controversial(x) & Damaged(x)))',
  'all x. (Skateboard(x) -> (SportsEquipment(x) & UsedForSkateboarding(x)))',
  'all x. (Skateboard(x) -> (Moves(x) & (Pushing(x) & Balanced(x)) | Pumping(x)))',
  'all x. (Train(x) -> (SeriesOfConnectedVehicles(x) & RunAlongRailwayTrack(x) & Transport(x)))',
  'all x. (Train(x) -> (RootsInWagonways(x) & UsedRailwayTracks(x) & PoweredByHorsesOrCables(x)))',
  'all x. (DeliveryHero(x) -> (GermanMultinational(x) & OnlineFoodOrdering(x) & FoodDelivery(x) & BasedInBerlin(x)))',
  'all x. (Delivery(x) -> (SingleAction(x) & Bowling(x) & TowardBatter(x)))',
  'all x. (CabellCalloway(x) -> (American(x) & JazzSinger(x) & Bandleader(x)))',
  'all x. (DeathCabForCutie(x) -> (American(x) &

### Conclusion

In [52]:
premises_prep = [utils.convert_to_nltk_rep(p) for p in premises_only[0]]
conclusion_prep = utils.convert_to_nltk_rep(conclusion_only)

try:
    # returns true, false, uncertain
    print(utils.evaluate(premises_only, conclusion_only)) 
except:
    # return syntax error
    print('Syntax error')

Uncertain


In [53]:
utils.evaluate(premises_only, conclusion_only)

'Uncertain'